# 作業

請使用 ImageDataGenerator 來進行 Cifar-10 資料集的訓練，並觀察不同的圖像增強方法是否會顯著影響訓練結果

In [1]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

Using TensorFlow backend.


In [8]:
# 讀取 Cifar-10 資料集
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
# Setting hyper-parameters
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練的 epochs 數量

In [9]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

from keras.utils import to_categorical
# Convert class vectors to binary class matrices.
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [5]:
import math
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
from keras.utils import to_categorical

class Model():
    def __init__(self, x_train, y_train, x_test, y_test):
        self.FirConvSize = None
        self.SecConvSize = None
        self.Padding = None
        self.model = None
        self.KernlSize = None
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        self.cnt = 0
    
    def setHypParam(self, padding='same', KernlSize=(3, 3), FirConvSize=32, SecConvSize=64):
        self.FirConvSize = FirConvSize
        self.SecConvSize = SecConvSize
        self.KernlSize = KernlSize
        self.Padding = padding
    
    def buildModel(self):
        self.model = Sequential()
        self.model.add(Conv2D(self.FirConvSize, self.KernlSize, padding=self.Padding,
                         input_shape=x_train.shape[1:]))
        self.model.add(Activation('relu'))
        self.model.add(Conv2D(self.FirConvSize, self.KernlSize))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.25))
        
        self.model.add(Conv2D(self.SecConvSize, self.KernlSize, padding=self.Padding))
        self.model.add(Activation('relu'))
        self.model.add(Conv2D(self.SecConvSize, self.KernlSize))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.25))

        self.model.add(Flatten())
        self.model.add(Dense(512))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(num_classes))
        self.model.add(Activation('softmax'))
    
    def setCompile(self):
        self.model.compile(loss='categorical_crossentropy',
                           optimizer=RMSprop(),
                           metrics=['accuracy'])
        
    def train_generator(self, batch_num = 32):
        while True:
            for indexs in range(0, len(self.x_train), batch_num):
                x_train_gen = self.x_train[indexs: indexs+batch_num]
                y_train_gen = self.y_train[indexs: indexs+batch_num]
                yield x_train_gen, y_train_gen
    
    def fitModel_generator(self):
        data_generator = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True)
        
        self.model.fit_generator(data_generator.flow(x_train, y_train, batch_size=32),
                                 steps_per_epoch=len(x_train) / 32,
                                 epochs=10)
    
    def evaluateScore(self):
        return (self.model.evaluate(self.x_test, self.y_test, verbose=0))

In [7]:
FirConvSize = 64
SecConvSize = 64
padding = 'same'
KernelSize_list = (3, 3)

model = Model(x_train, y_train, x_test, y_test)
model.setHypParam(FirConvSize=FirConvSize, SecConvSize=SecConvSize)
model.buildModel()
model.setCompile()
model.fitModel_generator()
score = model.evaluateScore()
print('Test loss:%.4f' %score[0])
print('Test accuracy:%.4f' %score[1])

Epoch 1/10
1563/1562 [==============================] - 25s 16ms/step - loss: 1.7838 - acc: 0.3449
Epoch 2/10
1563/1562 [==============================] - 25s 16ms/step - loss: 1.4773 - acc: 0.4713
Epoch 3/10
1563/1562 [==============================] - 26s 16ms/step - loss: 1.3952 - acc: 0.5059
Epoch 4/10
1563/1562 [==============================] - 25s 16ms/step - loss: 1.3727 - acc: 0.5176
Epoch 5/10
1563/1562 [==============================] - 32s 21ms/step - loss: 1.3795 - acc: 0.5240
Epoch 6/10
1563/1562 [==============================] - 34s 22ms/step - loss: 1.4034 - acc: 0.5161
Epoch 7/10
1563/1562 [==============================] - 29s 19ms/step - loss: 1.4275 - acc: 0.5130
Epoch 8/10
1563/1562 [==============================] - 31s 20ms/step - loss: 1.4920 - acc: 0.4965
Epoch 9/10
1563/1562 [==============================] - 29s 19ms/step - loss: 1.5301 - acc: 0.4805
Epoch 10/10
1563/1562 [==============================] - 23s 15ms/step - loss: 1.5713 - acc: 0.4668
Test loss